In [1]:
import numpy as np
np.set_printoptions(precision=2)

# Input

In [2]:
L_panel = 77.01 #  inches
W_panel = 39.02 #  inches
D_panel = 57.32 #  lbs
D_foot = 9 #  lbs
D_block = 34 #  lbs
μ = 0.49
γ_D = 0.6 #  ASD Dead Load Factor
γ_W = 0.6 #  ASD >2010 Wind Load Factor
γ_Wu = 1.6 #  LRFD >2010 Wind Load Factor
W_uplift_T = 962 #  lbf (total wind uplift for entire subarray)
W_drag_T = 358 #  lbf (total wind drag for entire subarray)

## PV Array Matrices

In [3]:
Mat_feet =  np.array([
                [1,0,1,0,1],
                [1,0,1,0,1],
                [1,0,1,0,1],
                [1,0,1,0,1],
                [1,0,1,0,1]
            ])
Mat_feet

array([[1, 0, 1, 0, 1],
       [1, 0, 1, 0, 1],
       [1, 0, 1, 0, 1],
       [1, 0, 1, 0, 1],
       [1, 0, 1, 0, 1]])

In [4]:
x_feet = np.array([[0.5*L_panel*j for j in range(len(row))] for row in Mat_feet])
x_feet #  inches

array([[   0.  ,   38.51,   77.01,  115.52,  154.02],
       [   0.  ,   38.51,   77.01,  115.52,  154.02],
       [   0.  ,   38.51,   77.01,  115.52,  154.02],
       [   0.  ,   38.51,   77.01,  115.52,  154.02],
       [   0.  ,   38.51,   77.01,  115.52,  154.02]])

In [5]:
Mat_panels = np.array([[1 for j in range((len(row)-1)//2)] for row in Mat_feet[:-1]])
Mat_panels

array([[1, 1],
       [1, 1],
       [1, 1],
       [1, 1]])

In [6]:
x_panels = np.array([[L_panel*(j+1-0.5) for j in range(len(row))] for row in Mat_panels])
x_panels #  inches

array([[  38.51,  115.52],
       [  38.51,  115.52],
       [  38.51,  115.52],
       [  38.51,  115.52]])

In [7]:
N_block =  2*Mat_feet
N_block

array([[2, 0, 2, 0, 2],
       [2, 0, 2, 0, 2],
       [2, 0, 2, 0, 2],
       [2, 0, 2, 0, 2],
       [2, 0, 2, 0, 2]])

## Dead Load Calculations

In [8]:
D_blocks = D_block*N_block
D_blocks

array([[68,  0, 68,  0, 68],
       [68,  0, 68,  0, 68],
       [68,  0, 68,  0, 68],
       [68,  0, 68,  0, 68],
       [68,  0, 68,  0, 68]])

In [9]:
def trib_area(i, j, Mat_feet):
    '''Calculate panel tributary area for a foot a location i,j
    in a foot matrix'''
    if Mat_feet[i,j] == 0:
        return 0 #  No foot at location
    if Mat_feet[i][j] not in (0, 1):
        raise ValueError('Invalid feet matrix') #  max 1 foot per location
    rowtrib = 1 if i in range(Mat_feet.shape[0])[1:-1] else 0.5
    if j == 0:
        return {0: rowtrib*0.5, 1: rowtrib*0.25}[Mat_feet[i][j+1]]
    if j == Mat_feet.shape[1]-1:
        return {0: rowtrib*0.5, 1: rowtrib*0.25}[Mat_feet[i][j-1]]
    result = {0: rowtrib*0.5, 1: rowtrib*0.25}[Mat_feet[i][j+1]]
    result += {0: rowtrib*0.5, 1: rowtrib*0.25}[Mat_feet[i][j-1]]
    return result

In [10]:
A_trib = np.array([[trib_area(i,j,Mat_feet) for j in range(len(row))] for i,row in enumerate(Mat_feet)])
A_trib

array([[ 0.25,  0.  ,  0.5 ,  0.  ,  0.25],
       [ 0.5 ,  0.  ,  1.  ,  0.  ,  0.5 ],
       [ 0.5 ,  0.  ,  1.  ,  0.  ,  0.5 ],
       [ 0.5 ,  0.  ,  1.  ,  0.  ,  0.5 ],
       [ 0.25,  0.  ,  0.5 ,  0.  ,  0.25]])

In [11]:
N_panels = A_trib.sum()
assert N_panels == Mat_panels.sum() #  Error if area calculation incorrect
N_panels

8.0

In [12]:
D_panels = A_trib*D_panel
D_panels #  lbs

array([[ 14.33,   0.  ,  28.66,   0.  ,  14.33],
       [ 28.66,   0.  ,  57.32,   0.  ,  28.66],
       [ 28.66,   0.  ,  57.32,   0.  ,  28.66],
       [ 28.66,   0.  ,  57.32,   0.  ,  28.66],
       [ 14.33,   0.  ,  28.66,   0.  ,  14.33]])

In [13]:
D_feet = D_foot*Mat_feet + D_panels + D_blocks
D_feet

array([[  91.33,    0.  ,  105.66,    0.  ,   91.33],
       [ 105.66,    0.  ,  134.32,    0.  ,  105.66],
       [ 105.66,    0.  ,  134.32,    0.  ,  105.66],
       [ 105.66,    0.  ,  134.32,    0.  ,  105.66],
       [  91.33,    0.  ,  105.66,    0.  ,   91.33]])

## Moment Calculations

#### Moment Due to Dead Load

In [14]:
M_zD = D_feet*x_feet #  in-lbs
M_zD

array([[     0.  ,      0.  ,   8136.88,      0.  ,  14066.65],
       [     0.  ,      0.  ,  10343.98,      0.  ,  16273.75],
       [     0.  ,      0.  ,  10343.98,      0.  ,  16273.75],
       [     0.  ,      0.  ,  10343.98,      0.  ,  16273.75],
       [     0.  ,      0.  ,   8136.88,      0.  ,  14066.65]])

In [15]:
M_zD_T = M_zD.sum() #  in-lbs
M_zD_T

124260.2556

In [16]:
r_zD = M_zD/M_zD_T
r_zD

array([[ 0.  ,  0.  ,  0.07,  0.  ,  0.11],
       [ 0.  ,  0.  ,  0.08,  0.  ,  0.13],
       [ 0.  ,  0.  ,  0.08,  0.  ,  0.13],
       [ 0.  ,  0.  ,  0.08,  0.  ,  0.13],
       [ 0.  ,  0.  ,  0.07,  0.  ,  0.11]])

In [17]:
r_zD.sum() #  correct if 1

1.0

#### Moment Due to Wind Uplift

In [18]:
W_uplift = W_uplift_T/N_panels #  lbs
W_uplift

120.25

In [19]:
M_zW = W_uplift*x_panels #  in-lbs
M_zW

array([[  4630.23,  13890.68],
       [  4630.23,  13890.68],
       [  4630.23,  13890.68],
       [  4630.23,  13890.68]])

In [20]:
M_zW_T = M_zW.sum() #  in-lbs
M_zW_T

74083.62000000001

In [21]:
M_z_distrib = M_zW_T*r_zD
M_z_distrib

array([[    0.  ,     0.  ,  4851.18,     0.  ,  8386.5 ],
       [    0.  ,     0.  ,  6167.05,     0.  ,  9702.37],
       [    0.  ,     0.  ,  6167.05,     0.  ,  9702.37],
       [    0.  ,     0.  ,  6167.05,     0.  ,  9702.37],
       [    0.  ,     0.  ,  4851.18,     0.  ,  8386.5 ]])

In [22]:
M_z_net = γ_D*M_zD-γ_W*M_z_distrib
M_z_net

array([[    0.  ,     0.  ,  1971.42,     0.  ,  3408.09],
       [    0.  ,     0.  ,  2506.16,     0.  ,  3942.83],
       [    0.  ,     0.  ,  2506.16,     0.  ,  3942.83],
       [    0.  ,     0.  ,  2506.16,     0.  ,  3942.83],
       [    0.  ,     0.  ,  1971.42,     0.  ,  3408.09]])

## Horizontal Friction Force and Moment

In [23]:
F_z_net = np.array([[moment/x if x != 0 else 0 for x,moment in zip(xrows, Mrows)] for xrows, Mrows in zip(x_feet, M_z_net)])
F_z_net

array([[  0.  ,   0.  ,  25.6 ,   0.  ,  22.13],
       [  0.  ,   0.  ,  32.54,   0.  ,  25.6 ],
       [  0.  ,   0.  ,  32.54,   0.  ,  25.6 ],
       [  0.  ,   0.  ,  32.54,   0.  ,  25.6 ],
       [  0.  ,   0.  ,  25.6 ,   0.  ,  22.13]])

In [24]:
F_res = μ*np.array([[F if F>0 else 0 for F in row] for row in F_z_net])
F_res

array([[  0.  ,   0.  ,  12.54,   0.  ,  10.84],
       [  0.  ,   0.  ,  15.95,   0.  ,  12.54],
       [  0.  ,   0.  ,  15.95,   0.  ,  12.54],
       [  0.  ,   0.  ,  15.95,   0.  ,  12.54],
       [  0.  ,   0.  ,  12.54,   0.  ,  10.84]])

In [25]:
M_res = F_res*x_feet
M_res

array([[    0.  ,     0.  ,   965.99,     0.  ,  1669.96],
       [    0.  ,     0.  ,  1228.02,     0.  ,  1931.99],
       [    0.  ,     0.  ,  1228.02,     0.  ,  1931.99],
       [    0.  ,     0.  ,  1228.02,     0.  ,  1931.99],
       [    0.  ,     0.  ,   965.99,     0.  ,  1669.96]])

In [26]:
M_res_T = M_res.sum()
M_res_T

14751.930866399996

## Wind Drag Force and Moment

In [27]:
W_drag = W_drag_T/N_panels #  lbs
W_drag

44.75

In [28]:
F_app = γ_W*W_drag*Mat_panels
F_app

array([[ 26.85,  26.85],
       [ 26.85,  26.85],
       [ 26.85,  26.85],
       [ 26.85,  26.85]])

In [29]:
M_app = F_app*x_panels
M_app

array([[ 1033.86,  3101.58],
       [ 1033.86,  3101.58],
       [ 1033.86,  3101.58],
       [ 1033.86,  3101.58]])

In [30]:
M_app_T = M_app.sum()
M_app_T

16541.748

## Excess Moment

In [31]:
M_excess = max(M_app_T - M_res_T, 0)
M_excess

1789.8171336000032

## Clamp Forces

In [32]:
N_rows = Mat_feet.shape[0]
N_rows

5

In [33]:
N_tension_clamp_rows = N_rows//2
N_tension_clamp_rows

2

In [34]:
N_clamps = np.array([1 if i == N_tension_clamp_rows-1 else 2 for i in range(N_tension_clamp_rows)])
N_clamps

array([2, 1])

In [35]:
CR_arm = np.array([i+0.5+N_rows/2-N_tension_clamp_rows for i in range(N_tension_clamp_rows)])
CR_arm

array([ 1.,  2.])

In [36]:
CR_arm1 = CR_arm[0]
CR_arm1

1.0

In [37]:
C_arm = N_clamps*CR_arm
C_arm

array([ 2.,  2.])

In [38]:
N_arms = sum(arm/CR_arm1 for arm in C_arm)
N_arms

4.0

In [39]:
F_couple = (1/N_arms)*M_excess/(CR_arm1*W_panel)
F_couple

11.467306084059476

In [40]:
F_clamp = 110 #  lbs

In [41]:
γ_Wu*F_couple

18.347689734495162

In [42]:
F_clamp>γ_Wu*F_couple #  OK if True

True

Required Ballast:

In [43]:
N_block

array([[2, 0, 2, 0, 2],
       [2, 0, 2, 0, 2],
       [2, 0, 2, 0, 2],
       [2, 0, 2, 0, 2],
       [2, 0, 2, 0, 2]])